# Step5: Post-Processing function step 2

In [1]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append("../../../GOSTnets")
import GOSTnets as gn

from shapely.geometry import LineString, Point

In [2]:
# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

In [3]:
from GOSTnets.load_traffic2 import *

In [4]:
# read graph
G = nx.read_gpickle('./sri_lanka_unclean2_w_time_largest_20200616.pickle')

In [5]:
len(G.edges)

1675316

In [6]:
# import shapefile as GPD
study_area = gpd.read_file("./airport_output_edges_20200628/airport_weighted_sec_saved_edges_all_merged.shp")

In [7]:
study_area

,o,d,w,sec_saved,imp_cost,mean_speed,length,infra_type,weighted_s,max_speed,min_speed,mode,osm_id,speed,time,geometry
0,634172331,3237361074,1,0.000000,0.000000,74.337302,0.061831,trunk,0.000000,NaN,NaN,None,NaN,NaN,NaN,"LINESTRING (80.49564 8.83425, 80.49552 8.83370)"
1,634172332,5930803566,1,0.000000,0.000000,61.887897,0.014707,trunk,0.000000,NaN,NaN,None,NaN,NaN,NaN,"LINESTRING (80.49947 8.85369, 80.49944 8.85356)"
2,634172333,3237354565,1,0.000000,0.000000,66.266369,0.048060,trunk,0.000000,NaN,NaN,None,NaN,NaN,NaN,"LINESTRING (80.50621 8.88353, 80.50611 8.88311)"
3,634172334,2408760333,1,0.000000,0.000000,71.781746,0.127612,trunk,0.000000,NaN,NaN,None,NaN,NaN,NaN,"LINESTRING (80.51146 8.92540, 80.51137 8.92425)"
4,634172335,6429885121,1,0.000000,0.000000,61.470238,0.005813,trunk,0.000000,NaN,NaN,None,NaN,NaN,NaN,"LINESTRING (80.52242 8.95585, 80.52241 8.95580)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18422,3206098680,3206098688,1,8.406632,1801.270959,27.974475,0.148602,secondary,8.406632,28.0,28.0,drive,50127324.0,28.000000,1.239786,"MULTILINESTRING ((80.00092 7.07288, 80.00085 7..."
18423,3206095326,636968555,1,5.421380,2226.852778,29.773254,0.113648,secondary,5.421380,36.0,30.0,drive,50127324.0,30.074405,1.426987,"MULTILINESTRING ((80.00945 7.06452, 80.00936 7..."
18424,3213647870,5647606483,1,2.377077,14814.738742,43.031132,0.242110,secondary,2.377077,44.0,44.0,drive,321353841.0,44.000000,6.488838,"MULTILINESTRING ((80.49798 7.56449, 80.49870 7..."
18425,3498784687,5780895828,3,0.801155,2517.275355,18.166076,0.020771,tertiary,2.403465,28.0,28.0,drive,49047150.0,28.000000,1.732600,"MULTILINESTRING ((79.89322 6.83782, 79.89330 6..."


In [8]:
sorted_study_area = study_area.sort_values(by=['weighted_s'], ascending=False)

In [9]:
sorted_study_area[:15]

,o,d,w,sec_saved,imp_cost,mean_speed,length,infra_type,weighted_s,max_speed,min_speed,mode,osm_id,speed,time,geometry
18369,3169843787,4719012497,1,190.952134,3136.794673,28.972629,5.593547,tertiary,190.952134,29.0,29.0,drive,513498108.0,29.000000,2.084557,"MULTILINESTRING ((81.07437 6.83304, 81.07450 6..."
18409,5647606451,4982371638,1,107.780874,1446.278072,39.285160,5.488517,secondary,107.780874,42.0,2.0,drive,371237411.0,39.285218,0.709493,"MULTILINESTRING ((80.55331 7.57692, 80.55324 7..."
18417,3069814342,3450046074,1,52.834367,5239.061357,28.658097,1.013358,secondary,52.834367,29.0,29.0,drive,503548865.0,29.000000,3.481618,"MULTILINESTRING ((80.55149 7.56911, 80.55156 7..."
18406,3069857983,3069857968,1,52.776129,9253.986102,34.970874,1.865071,secondary,52.776129,41.0,28.0,drive,199357951.0,35.893353,4.968673,"MULTILINESTRING ((80.45938 7.56123, 80.45981 7..."
18375,5181016959,2035876118,3,16.204121,567.330965,12.720707,0.079074,secondary,48.612363,13.0,13.0,drive,22362995.0,13.000000,0.841044,"MULTILINESTRING ((79.92590 6.84421, 79.92589 6..."
18374,1737485314,2068144084,3,11.413893,2174.870295,15.079983,0.068535,secondary,34.241678,35.0,3.0,drive,131349571.0,15.091766,2.777275,"MULTILINESTRING ((79.90074 6.84063, 79.90084 6..."
18389,5831722223,3800021436,3,10.820891,1746.295737,21.053847,0.133617,tertiary,32.462673,24.0,18.0,drive,131349578.0,21.054563,1.598443,"MULTILINESTRING ((79.88892 6.83714, 79.88885 6..."
18397,1679736164,2036049401,3,10.670293,3424.276751,20.757678,0.131000,tertiary,32.010878,24.0,19.0,drive,131349578.0,20.997024,3.142945,"MULTILINESTRING ((79.88618 6.83650, 79.88601 6..."
18390,5831686996,2068144055,3,9.620130,1631.454226,20.788198,0.117878,tertiary,28.860389,23.0,19.0,drive,131349578.0,20.977679,1.498798,"MULTILINESTRING ((79.88792 6.83683, 79.88784 6..."
10686,1737485290,3498670709,3,9.068678,5574.918183,9.577877,0.029844,secondary,27.206034,NaN,NaN,None,NaN,NaN,NaN,"LINESTRING (79.90031 6.84051, 79.90056 6.84059)"


In [10]:
total_cost = 0
total_count = 0

In [11]:
def sort_weighted_s(sec_saved, imp_cost):
    global total_cost
    global total_count
    if total_cost < 20000000:
        print(f"total_cost before:{total_cost}")
        print(f"imp_cost:{imp_cost}")
        total_cost = total_cost + imp_cost
        total_count += 1
        print(f"sec_saved:{sec_saved}")
        print(f"total_cost:{total_cost}")
        print(f"total_count:{total_count}")


In [12]:
#sorted_study_area['weighted_s'] = 
sorted_study_area.apply(lambda x: sort_weighted_s(x['sec_saved'],x['imp_cost']),axis=1)

total_cost before:0
imp_cost:3136.794673091688
sec_saved:190.95213441526755
total_cost:3136.794673091688
total_count:1
total_cost before:3136.794673091688
imp_cost:1446.2780723870906
sec_saved:107.78087377545062
total_cost:4583.072745478778
total_count:2
total_cost before:4583.072745478778
imp_cost:5239.061357340168
sec_saved:52.83436686543669
total_cost:9822.134102818945
total_count:3
total_cost before:9822.134102818945
imp_cost:9253.98610197961
sec_saved:52.77612861508081
total_cost:19076.120204798557
total_count:4
total_cost before:19076.120204798557
imp_cost:567.330965370513
sec_saved:16.204120960896386
total_cost:19643.45117016907
total_count:5
total_cost before:19643.45117016907
imp_cost:2174.8702949451035
sec_saved:11.413892630351867
total_cost:21818.321465114175
total_count:6
total_cost before:21818.321465114175
imp_cost:1746.2957372126261
sec_saved:10.820891046304613
total_cost:23564.6172023268
total_count:7
total_cost before:23564.6172023268
imp_cost:3424.276750783219
sec_sav

total_count:408
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:409
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:410
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:411
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:412
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:413
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:414
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:415
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:416
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_coun

total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:685
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:686
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:687
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:688
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:689
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:690
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:691
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:692
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:693
total_cost

total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1056
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1057
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1058
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1059
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1060
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1061
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1062
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1063
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1064
t

imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1423
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1424
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1425
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1426
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1427
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1428
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1429
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1430
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:1431
total_cost before:2265150.3609043043
i

...

total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17323
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17324
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17325
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17326
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17327
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17328
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17329
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17330
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_coun

total_count:17686
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17687
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17688
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17689
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17690
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17691
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17692
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17693
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:17694
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.360

total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:18047
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:18048
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:18049
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:18050
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:18051
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:18052
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:18053
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_count:18054
total_cost before:2265150.3609043043
imp_cost:0.0
sec_saved:0.0
total_cost:2265150.3609043043
total_coun

total_cost before:3647193.875879162
imp_cost:17554.921291866704
sec_saved:-6.766350819134918
total_cost:3664748.797171029
total_count:18424
total_cost before:3664748.797171029
imp_cost:28700.53548104236
sec_saved:-3.687434965444414
total_cost:3693449.332652071
total_count:18425
total_cost before:3693449.332652071
imp_cost:23036.385235817528
sec_saved:-8.312368265874293
total_cost:3716485.7178878887
total_count:18426
total_cost before:3716485.7178878887
imp_cost:22289.919204077076
sec_saved:-8.92184214559501
total_cost:3738775.637091966
total_count:18427


18369    None
18409    None
18417    None
18406    None
18375    None
         ... 
151      None
2095     None
10044    None
2125     None
152      None
Length: 18427, dtype: object

In [13]:
total_cost

3738775.637091966

In [14]:
total_count

18427